<font color="white">.</font> | <font color="white">.</font> | <font color="white">.</font>
-- | -- | --
![NASA](http://www.nasa.gov/sites/all/themes/custom/nasatwo/images/nasa-logo.svg) | <h1><font size="+3">ASTG Python Courses</font></h1> | ![NASA](https://www.nccs.nasa.gov/sites/default/files/NCCS_Logo_0.png)

---

<CENTER>
<H1 style="color:red">
Accelerating (Numba) and Scaling (Dask) Python Codes on GPUs
</H1>
</CENTER>

## <font color='red'>Reference Documents</font>

- [An introduction to CUDA in Python (Part 1)](https://www.vincent-lunot.com/post/an-introduction-to-cuda-in-python-part-1/)
- [Supported Python features in CUDA Python](http://numba.pydata.org/numba-doc/latest/cuda/cudapysupported.html)
- <a href="https://nyu-cds.github.io/python-numba/05-cuda/">Introduction to Numba: CUDA Programming</a>
- <a href="https://people.duke.edu/~ccc14/sta-663/CUDAPython.html">Massively parallel programming with GPUs</a>
- <a href="https://www.kdnuggets.com/2019/07/accelerate-data-science-on-gpu.html">Here’s how you can accelerate your Data Science on GPU</a>
- [Numba for CUDA GPUs](http://numba.pydata.org/numba-doc/0.30.0/cuda/index.html)
- <a href="https://colab.research.google.com/github/evaneschneider/parallel-programming/blob/master/COMPASS_gpu_intro.ipynb">Introduction to GPU programming with Numba</a>
- <a href="https://thedatafrog.com/en/articles/make-python-fast-numba/">Make python fast with numba</a>
- <a href="https://www.deeplearningwizard.com/deep_learning/production_pytorch/speed_optimization_basics_numba/">Speed Optimization Basics: Numba</a>

## <font color='red'>What will be Covered?</font>

* Quick Overview of GPUs
* Numba on GPUs
   * Quick Overview of Numba
   * Basic Use of GPUs with Numba Decorators
   * Writing Cuda Kernel
   * Examples
* Dask-CuDF through Examples

# <font color='red'>Initial Setup</font>

## Accessing the GPU on Google Colab

In order to access GPUs for free:

1. Go to the `Runtime` menu,
2. Click on `Change runtime type`, and 
3. In the pop-up box, under `Hardware accelerator`, select `GPU` and click on `SAVE`.

## Environment Sanity Check ##

- <font color='red'>Click the _Runtime_ dropdown at the top of the page, then _Change Runtime Type_ and confirm the instance type is _GPU_.</font>
- Check the output of `!nvidia-smi` to make sure you've been allocated a Tesla T4.

In [ ]:
!nvidia-smi

#### Verify that you were allocated the GPU compatible with RAPIDS

In [ ]:
import pynvml

pynvml.nvmlInit()
handle = pynvml.nvmlDeviceGetHandleByIndex(0)
gpu_name = pynvml.nvmlDeviceGetName(handle).decode('UTF-8')

if('K80' not in gpu_name):
   print('***********************************************************************')
   print('Woo! Your instance has the right kind of GPU, a '+ str(gpu_name)+'!')
   print('***********************************************************************')
   print()
else:
   raise Exception("""
                  Unfortunately Colab didn't give you a RAPIDS compatible GPU (P4, P100, T4, or V100), but gave you a """+ gpu_name +""".
                  Make sure you've configured Colab to request a GPU Instance Type.                
                  If you get an incompatible GPU (i.e., a K80), use 'Runtime -> Factory Reset Runtimes...' to try again"""
                  )

#### Accessing CUDA Libraries

Even though the CUDA libararies, they might not be accessible. We need to find out where they are and include them in our Colab environment.

In [ ]:
import os

dev_lib_path = !find / -iname 'libdevice'
nvvm_lib_path = !find / -iname 'libnvvm.so'

assert len(dev_lib_path)>0, "Device Lib Missing"
assert len(nvvm_lib_path)>0, "NVVM Missing"

os.environ['NUMBAPRO_LIBDEVICE'] = dev_lib_path[0]
os.environ['NUMBAPRO_NVVM'] = nvvm_lib_path[0]

# <font color="red"> GPUs</font>

![GPUs](http://www.nvidia.com/docs/IO/143716/how-gpu-acceleration-works.png)
Image Source: NVIDIA

- Graphics Processing Units (GPUs) are custom designed to be very efficient at handling computer graphics and image processing.
- Central Processing Units (CPUs) handle computations serially, meaning the logic in handled in one stream: the next task will complete when the subsequent task has finished. CPUs can execute tasks in parallel across cores. For example, most computer CPUs tend to have either two, four or six cores.
- In comparison, GPUs have hundreds of 'cores'. This massively parallel architecture is what gives the GPU its high compute performance.

**Useful Terminology**

| Term | Meaning |
| ---  | --- |
| `host` | the CPU |
| `device` | the GPU |
| `host memory` | the system main memory |
| `device memory` | onboard memory on a GPU card |
| `kernels` | a GPU function launched by the host and executed on the device |
| `device function` | a GPU function executed on the device which can only be called from the device  |


### Numba and GPUs

- Numba supports CUDA GPU programming by directly compiling a restricted subset of Python code into CUDA kernels and device functions following the CUDA execution model. 
- Kernels written in Numba appear to have direct access to NumPy arrays. 
- NumPy arrays are transferred between the CPU and the GPU automatically.

### Dask and GPUs

- Dask can help to scale out large array and dataframe computations by combining the Dask Array and DataFrame collections with a GPU-accelerated array or dataframe library.
- The RAPIDS libraries provide a GPU accelerated Pandas-like library, `cuDF`, which interoperates well and is tested against Dask DataFrame.
- You can convert a Pandas-backed Dask DataFrame to a cuDF-backed Dask DataFrame.

![rapids](https://pbs.twimg.com/media/D2CeyaYVAAAe3kM.jpg)
Image Source: NVIDIA

---

# <font color="red"> CUDA Programming</font>

- A GPU program comprises two parts:
   - A host part that runs on the CPU, and
   - One or more kernels that run on the GPU.
- Typically, the CPU portion of the program is used to set up the parameters and data for the computation, while the kernel portion performs the actual computation.
- In some cases the CPU portion may comprise a parallel program that performs message passing operations using MPI.
- CUDA Python maps directly to the single-instruction multiple-thread execution (SIMT) model of CUDA. 
- Each instruction is implicitly executed by multiple threads in parallel. 
- Array expressions are less useful because we don’t want multiple threads to perform the same task. Instead, we want threads to perform a task in a cooperative fashion.




---

# <font color='red'>Numba</font>

> Numba is an open-source JIT compiler that translates a subset of Python and NumPy into fast machine code using `LLVM` (low-level virtual machine), via the llvmlite Python package. It offers a range of options for parallelising Python code for CPUs and GPUs, often with only minor code changes. 
>
>Wikipedia

The diagram below, shows all the steps carried out by Numba to execute `do_math`. 

![fig_numba](https://miro.medium.com/max/1400/1*S0S4QUjR-BsdTICtT9797Q.png)
Image Source: Continuum Analytics

- **IR**: Intermediate Representations
- **Bytecode Analysis**: Intermediate code more abstract than machine code
- **LLVM**: Low Level Virtual Machine, infrastructure to develop compilers
- **NVVM**: It is an IR compiler based on LLVM, it is designed to represent GPU kernels


    
**Usage**:

- Numba provides several utilities for code generation.
- Its central feature is the `numba.jit()` decorator. 
- Using this decorator, you can mark a function for optimization by Numba’s JIT compiler. - - - Various invocation modes trigger differing compilation options and behaviours.


In [1]:
import time
import numpy as np
import numba as nb
from numba import jit
from numba import guvectorize
from numba import vectorize
from numba import njit
from numba import prange
from numba import cuda

**Checking your System**

The `numba -s` or `numba --sysinfo` command prints a lot of information about your system and your Numba installation and relevant dependencies.

In [ ]:
!numba -s

In [ ]:
print(cuda.gpus)

## <font color='blue'>Basic Usage of Numba on GPUs</font>

###  <font color="green">Example:</font> Average of Square Root of Entries of an Array

In [ ]:
import math

def average_sqrt_1D(array1D):
    """
        Average of the square root of all the entries of 
        an array using loops.
    """
    avg = 0.
    for x in array1D:
        avg += math.sqrt(x)
    return avg/len(array1D)

In [ ]:
@njit
def average_sqrt_1D_numba(array1D):
    """
        Average of the square root of all the entries of 
        an array using loops.
    """
    avg = 0.
    for x in array1D:
        avg += math.sqrt(x)
    return avg/len(array1D)

#### Other ways of doing the calculations

In [ ]:
def average_sqrt_1D_2(array1D):
    return np.sum(np.sqrt(array1D))/array1D.size

In [ ]:
@guvectorize(['(float64[:], float64[:])'], '(m)->()')
def average_sqrt_1D_numba_2(array1D, avg):
    tmp = 0.
    for x in array1D:
        tmp += math.sqrt(x)
    avg = tmp/len(array1D)

#### First use of GPUs

There are two ways to use Numba on GPUs:
- The first way uses ufuncs or gufuncs (GPU universal functions). 
- In the second way, you define CUDA Python kernels. 

To use the ufuncs/gufuncs approach:
- We need to use the Numba `guvectorize` decorator:
    - The functions it decorates, work with an arbitrary number of elements of input arrays, and take and return arrays of differing dimensions. 
    - The function do not return their result value: they take it as an array argument, which must be filled in by the function. 
    - Requires a signature for declaring an input and output layouts, in symbolic form. For instance `(n),()->(n)` tells NumPy that the function takes a n-element one-dimension array, a scalar (symbolically denoted by the empty tuple `()`) and returns a n-element one-dimension array
- We specify the `cuda` target as argument of the decorator. 


Numba will automatically:

- Compile a Cuda kernel to execute the operations.
- Allocate GPU memory for the input.
- Execute the CUDA kernel with the correct kernel dimensions given the input sizes.
- Copy the result back from the GPU to the CPU.
- Return the result on the host.

In [ ]:
@guvectorize(['(float64[:], float64[:])'], '(n)->()', target ="cuda")
def average_sqrt_1D_cuda(array1D, avg):
    """
        Average of the square root of all the entries of 
        an array using loops.
    """
    tmp = 0.
    for x in array1D:
        tmp += math.sqrt(x)
    avg[0] = tmp/len(array1D)

In [ ]:
M = 10000000
array1D = np.random.rand(M)

In [ ]:
avg = average_sqrt_1D(array1D)
print(avg)

In [ ]:
avg = average_sqrt_1D_numba(array1D)
print(avg)

In [ ]:
avg = average_sqrt_1D_cuda(array1D)
print(avg)

In [ ]:
time_reg = %timeit -o average_sqrt_1D(array1D)

In [ ]:
time_numba = %timeit -o average_sqrt_1D_numba(array1D)

In [ ]:
time_cuda1 = %timeit -o average_sqrt_1D_cuda(array1D)

In [ ]:
print("Regular Speedup:     {}".format(time_reg.best/time_reg.best))
print("Numba   Speedup:     {}".format(time_reg.best/time_numba.best))
print("Cuda    Speedup:     {}".format(time_reg.best/time_cuda1.best))

## <font color="blue">Writing Cuda Kernels</font>

- In CUDA, the code you write will be executed by multiple threads at once (often hundreds or thousands). 
- The CUDA programming model allows you to abstract the GPU hardware into a software model defined by a thread hierachy that is composed of a **grid** containing **blocks** of **threads**. 
- These **threads** are the smallest individual unit in the programming model, and they execute together in groups (traditionally called **warps**, consisting of 32 threads each). 

### <font color="violet">Kernel Declaration</font>

A kernel function is a GPU function that is meant to be called from CPU code. It has two fundamental characteristics:

- **Kernels cannot explicitly return a value**; all result data must be written to an array passed to the function (if computing a scalar, you will probably pass a one-element array).
- **Kernels explicitly declare their thread hierarchy when called**: i.e. the number of thread blocks and the number of threads per block (note that while a kernel is compiled once, it can be called multiple times with different block sizes or grid sizes).
- To tell Python that a function is a CUDA kernel, simply add `@cuda.jit` before the definition. 

```python
@cuda.jit
def average_sqrt_1D_cuda_kernel(array1D, avg):
    """
    Code for kernel.
    """
    # code here
```


### <font color="violet">Kernel Invocation</font>

There are two main steps:

- Instantiate the kernel properly, by specifying a **number of blocks per grid** and a **number of threads per block**. 
    - The product of the two will give the total number of threads launched. 
    - Kernel instantiation is done by taking the compiled kernel function (here `average_sqrt_1D_cuda_kernel`) and indexing it with a tuple of integers.
- Running the kernel, by passing it the input array (and any separate output arrays if necessary). 
    - By default, running a kernel is synchronous: the function returns when the kernel has finished executing and the data is synchronized back.

A kernel is typically launched in the following way:

```python
# Create the data array - usually initialized some other way
array1D = numpy.ones(256)

avg = 0.0

# Set the number of threads in a block
threads_per_block = 32 

# Calculate the number of thread blocks in the grid
blocks_per_grid = (array1D.size + (threads_per_block - 1)) // threads_per_block

# Now start the kernel
average_sqrt_1D_cuda_kernel[blocks_per_grid, threads_per_block](array1D, avg)

# Print the result
print(avg)
```


### <font color="violet">Choosing the Block Size</font>

The two-level thread hierarchy is important for the following reasons:

- On the software side, the block size determines how many threads share a given area of shared memory.
- On the hardware side, the block size must be large enough for full occupation of execution units; recommendations can be found in the [CUDA C Programming Guide](https://docs.nvidia.com/cuda/cuda-c-programming-guide/).

The block size you choose depends on a range of factors, including:

- The size of the data array
- The size of the shared memory per block (e.g. 64KB)
- The maximum number of threads per block supported by the hardware (e.g. 512 or 1024)
- The maximum number of threads per multiprocessor (MP) (e.g. 2048)
- The maximum number of blocks per MP (e.g. 32)
- The number of threads that can be executed concurrently (a “warp” i.e. 32).


Determiming the best size for your grid of thread blocks is a complicated problem that often depends on the specific algorithm and hardware you're using. Here a few good rules of thumb:

- The size of a block should be a multiple of 32 threads, with typical block sizes between 128 and 512 threads per block.
- **The size of the grid should ensure the full GPU is utilized where possible**. Launching a grid where the number of blocks is 2x-4x the number of **streaming multiprocessors** on the GPU is a good starting place. (The Tesla K80 GPUs provided by Colaboratory have 15 SMs - more modern GPUs like the P100s on TigerGPU have 60+.)
- The CUDA kernel launch overhead does depend on the number of blocks, so it may not be best to launch a grid where the number of threads equals the number of input elements when the input size is very big. 


### <font color="violet">Thread Positioning</font>
- When running a kernel, the kernel function’s code is executed by every thread once. 
- It therefore has to know which thread it is in, in order to know which array element(s) it is responsible for (complex algorithms may define more complex responsibilities, but the underlying principle is the same).
- One way is for the thread to determines its position in the grid and block and manually compute the corresponding array position.
- Blocks are organized into a one-dimensional, two-dimensional, or three-dimensional grid of thread blocks.

In [ ]:
@cuda.jit
def average_sqrt_1D_cuda_kernel(array1D, avg):
    # this is the unique thread ID within a 1D block
    tidx = cuda.threadIdx.x 
    # Similarly, this is the unique block ID within the 1D grid
    bidx = cuda.blockIdx.x  

    # number of threads per block
    block_dimx = cuda.blockDim.x 
    # number of blocks in the grid
    grid_dimx = cuda.gridDim.x    
    
    start = tidx + bidx * block_dimx
    stride = block_dimx * grid_dimx
    
    tmp = 0.
    for i in range(start, array1D.shape[0], stride):
        tmp += math.sqrt(array1D[i])
    avg = tmp/array1D.shape[0]

- `threadIdx`, `blockIdx`, `blockDim` and `gridDim` are special objects provided by the CUDA backend for the sole purpose of knowing the geometry of the thread hierarchy and the position of the current thread within that geometry.
- These objects can be 1D, 2D or 3D, depending on how the kernel was invoked. To access the value at each dimension, use the x, y and z attributes of these objects, respectively.
   - `numba.cuda.threadIdx`: The thread indices in the current thread block. For 1D blocks, the index (given by the x attribute) is an integer spanning the range from 0 inclusive to numba.cuda.blockDim exclusive. A similar rule exists for each dimension when more than one dimension is used.
   - `numba.cuda.blockDim`: The shape of the block of threads, as declared when instantiating the kernel. This value is the same for all threads in a given kernel, even if they belong to different blocks (i.e. each block is “full”).
   - `numba.cuda.blockIdx`: The block indices in the grid of threads launched a kernel. For a 1D grid, the index (given by the x attribute) is an integer spanning the range from 0 inclusive to numba.cuda.gridDim exclusive. A similar rule exists for each dimension when more than one dimension is used.
   - `numba.cuda.gridDim`: The shape of the grid of blocks, i.e. the total number of blocks launched by this kernel invocation, as declared when instantiating the kernel.

In [ ]:
threads_per_block = 32
blocks_per_grid = (array1D.size + (threads_per_block - 1)) // threads_per_block

In [ ]:
average_sqrt_1D_cuda_kernel[blocks_per_grid, threads_per_block](array1D, avg)
print(avg)

In [ ]:
time_cuda2 = %timeit -o average_sqrt_1D_cuda_kernel[blocks_per_grid, threads_per_block](array1D, avg)

**The execution time includes the transfer of the data from the host memory to the device memory before the computation and the transfer from the device memory back to the host memory at the end of the computation.**

In [ ]:
print("Regular Speedup:     {}".format(time_reg.best/time_reg.best))
print("Numba   Speedup:     {}".format(time_reg.best/time_numba.best))
print("Cuda    Speedup:     {}".format(time_reg.best/time_cuda1.best))
print("Cuda Kernel Speedup: {}".format(time_reg.best/time_cuda2.best))

## <font color='violet'>Absolute Positions</font>

- Simple algorithms will tend to always use thread indices in the same way as shown in the example above. 
- Numba provides additional facilities to automate such calculations:
  - `numba.cuda.grid`(ndim): Return the absolute position of the current thread in the entire grid of blocks. `ndim` should correspond to the number of dimensions declared when instantiating the kernel. If `ndim` is 1, a single integer is returned. If `ndim` is 2 or 3, a tuple of the given number of integers is returned.
  - `numba.cuda.gridsize`(ndim): Return the absolute size (or shape) in threads of the entire grid of blocks. `ndim` has the same meaning as in grid() above.
  
`cuda.grid()` is a convenience function provided by Numba. In CUDA, blocks and grids are actually three dimensional. 
- Each block has dimensions: `cuda.blockDim.x`, `cuda.blockDim.y`, and `cuda.blockDim.z`.
- The grid has dimensions: `cuda.gridDim.x`, `cuda.gridDim.y`, and `cuda.gridDim.z`.

This means that each block has:

$$num\_of\_threads\_per\_block = cuda.blockDim.x \times cuda.blockDim.y \times cuda.blockDim.z$$

And the grid has:

$$number\_of\_blocks = cuda.gridDim.x \times cuda.gridDim.y \times cuda.gridDim.z$$


The generic way to define the number of threads while launching a kernel is actually:

```python
kernel_name[(griddimx, griddimy, griddimz), (blockdimx, blockdimy, blockdimz)](arguments)
```
Launching a kernel specifying only two integers like, `cudakernel1[1024, 1024](array)`, is equivalent to launching a kernel with y and z dimensions equal to 1, e.g. `cudakernel1[(1024, 1, 1), (1024, 1, 1)](array)`.

CUDA provides the following values to identify each thread:

- `cuda.threadIdx.x`, `cuda.threadIdx.y`, `cuda.threadIdx.z` that give the (x, y, z) positions of the current thread inside the current block,
- `cuda.blockIdx.x`, `cuda.blockIdx.y`, `cuda.blockIdx.z` that give the (x, y, z) positions of the current block inside the grid.

Therefore, the absolute position of a thread inside the grid is given by:

```python
(cuda.blockIdx.x * cuda.blockDim.x + cuda.threadIdx.x, 
 cuda.blockIdx.y * cuda.blockDim.y + cuda.threadIdx.y,
 cuda.blockIdx.z * cuda.blockDim.z + cuda.threadIdx.z) 
```

The convenience function:
- `cuda.grid(3`) returns this whole tuple
- `cuda.grid(2)` returns the tuple (`cuda.blockIdx.x * cuda.blockDim.x + cuda.threadIdx.x`, `cuda.blockIdx.y * cuda.blockDim.y + cuda.threadIdx.y`) 
- `cuda.grid(1)` returns the integer `cuda.blockIdx.x * cuda.blockDim.x + cuda.threadIdx.x`.

![GPU_grid](https://www.researchgate.net/profile/Omar-Bouattane/publication/321666991/figure/fig2/AS:572931245260800@1513608861931/Figure-2-Execution-model-of-a-CUDA-program-on-NVidias-GPU-Hierarchy-grid-blocks-and_W640.jpg)
Image Source: Omar Bouttane

With these functions, the incrementation example can become:

![grid_1d](https://www.programmersought.com/images/863/2c49da674a6e9cc10d349b3e492428e7.png)
Image Source: programmersought.com

In [ ]:
@cuda.jit   
def average_sqrt_1D_cuda_kernel2(array1D, avg):
    # number of threads per block
    block_dimx = cuda.blockDim.x 
    # number of blocks in the grid
    grid_dimx = cuda.gridDim.x    
    
    start = cuda.grid(1)
    stride = block_dimx * grid_dimx
    
    tmp = 0.
    for i in range(start, array1D.shape[0], stride):
        tmp += math.sqrt(array1D[i])
    avg[0] = tmp/array1D.shape[0]

threads_per_block = 32
blocks_per_grid = (array1D.size + (threads_per_block - 1)) // threads_per_block

**We first transfer all the data to the device using the function `cuda.to_device` and we next execute the kernel with the device data as arguments:**

In [ ]:
d_array1D = cuda.to_device(array1D)
d_avg = cuda.device_array((1,))

In [ ]:
average_sqrt_1D_cuda_kernel2[blocks_per_grid, threads_per_block](d_array1D, d_avg)

In [ ]:
avg = d_avg.copy_to_host()
print(avg)

In [ ]:
time_cuda3 = %timeit -o average_sqrt_1D_cuda_kernel2[blocks_per_grid, threads_per_block](d_array1D, d_avg)

In [ ]:
print("Regular Speedup:       {}".format(time_reg.best/time_reg.best))
print("Numba   Speedup:       {}".format(time_reg.best/time_numba.best))
print("Cuda    Speedup:       {}".format(time_reg.best/time_cuda1.best))
print("Cuda Kernel   Speedup: {}".format(time_reg.best/time_cuda2.best))
print("Cuda Kernel 2 Speedup: {}".format(time_reg.best/time_cuda3.best))

### <font color="violet">Recommendations</font>
To achieve the best performance:

- Find ways to parallelize sequential code
- Minimize data transfers between the host and the device
- Adjust kernel launch configuration to maximize device utilization
- Ensure global memory accesses are coalesced
- Minimize redundant accesses to global memory whenever possible
- Avoid different execution paths within the same warp.

## Exercise:

Modify the code below so that it can run on GPUs.

In [ ]:
def increment_1D_array(inarray, val):
    outarray = np.zeros_like(inarray)
    for i in range(inarray.size):
        outarray[i] = inarray[i] + val
    return outarray

<details><summary><b><font color="violet" size=4>Click here to access the solution</font> </b></summary>
<p>

<b>With Numba njit</b>
```python
    @njit
    def increment_1D_array_numba1(inarray, val):
        outarray = np.zeros_like(inarray)
        for i in range(inarray.size):
            outarray[i] = inarray[i] + val
        return outarray
```

</p>

<p>
    
<b>With Numba vectorize</b>
```python
    @vectorize(['float64(float64, float64)'])
    def increment_1D_array_numba2(inarray, val):
        return inarray + val
```
    
</p>
<p>

<b>With Numba guvectorize</b>

```python
    @guvectorize(['(float64[:], float64, float64[:])'], '(n), ()->(n)', target ="parallel")
    def increment_1D_array_numba3(inarray, val, outarray):
        for i in range(inarray.size):
            outarray[i] = inarray[i] + val
```

</p>
    
    
<p>

<b>With Numba guvectorize and Cuda</b>
```python
    @guvectorize(['(float64[:], float64, float64[:])'], '(n), ()->(n)', target ="cuda")
    def increment_1D_array_cuda1(inarray, val, outarray):
        for i in range(inarray.size): 
            outarray[i] = inarray[i] + val
```

</p>

    
<p>

<b>With cuda.jit</b>
```python
    @cuda.jit
    def increment_1D_array_cuda2(inarray, val, outarray):  
        index = cuda.grid(1)
        if index < inarray.size: 
            outarray[index] = inarray[index] + val
```

</p>
    
```python

M = 100000000
inarray = np.random.rand(M)
outarray = np.zeros_like(inarray)
val = 2.75

time_reg = %timeit -o increment_1D_array(inarray, val)
time_numba1 = %timeit -o increment_1D_array_numba1(inarray, val)
time_numba2 = %timeit -o increment_1D_array_numba2(inarray, val)
time_numba3 = %timeit -o increment_1D_array_numba3(inarray, val, outarray)
time_cuda1 = %timeit -o increment_1D_array_cuda1(inarray, val, outarray)

threads_per_block = 32
blocks_per_grid = (inarray.size + (threads_per_block - 1)) // threads_per_block

time_cuda2 = %timeit -o increment_1D_array_cuda2[blocks_per_grid, threads_per_block](inarray, val, outarray)
    
    
print("Speedup Regular: {}".format(time_reg.best/time_reg.best))
print("Speedup Numba 1: {}".format(time_reg.best/time_numba1.best))
print("Speedup Numba 2: {}".format(time_reg.best/time_numba2.best))
print("Speedup Numba 3: {}".format(time_reg.best/time_numba3.best))
print("Speedup  Cuda 1: {}".format(time_reg.best/time_cuda1.best))
print("Speedup  Cuda 2: {}".format(time_reg.best/time_cuda2.best))
```
    
</details>

### <font color="green">Example:</font> Matrix Multiplication

In [ ]:
N = 256
A = np.random.rand(N, N)
B = np.random.rand(N, N)
C = np.zeros_like(A)

In [ ]:
def matrix_multiplication(A, B, C):
    """
        Perform square matrix multiplication of C = A * B using loops.
    """
    n = len(A[0])
    for i in range(n):
        for j in range(n):
            tmp = 0.
            for k in range(n):
                tmp  += A[i, k]*B[k, j]
            C[i, j] = tmp

In [ ]:
tRegMat = %timeit -o matrix_multiplication(A, B, C)

In [ ]:
@njit(parallel=True)
def matrix_multiplication_numba(A, B, C):
    """
        Perform square matrix multiplication of C = A * B using loops.
    """
    n = len(A[0])
    for i in prange(n):
        for j in prange(n):
            tmp = 0.
            for k in prange(n):
                tmp += A[i, k]*B[k, j]
            C[i,j] = tmp

In [ ]:
tNumMat = %timeit -o matrix_multiplication_numba(A, B, C)

#### First GPU Kernel
- Each thread reads one row of `A` and one column of `B` and computes the corresponding element of `C`. 
- For input arrays where `A.shape == (m, n)` and `B.shape == (n, p)` then the result shape will be `C.shape = (m, p)`.

![matmult1](https://nyu-cds.github.io/python-numba/fig/05-matmul.png)
Image Source: nyu-cds.github.io

In [ ]:
@cuda.jit
def matrix_multiplication_cuda(A, B, C):
    """
      Perform square matrix multiplication of C = A * B using loops.
    """
    row, col = cuda.grid(2)
    if row < C.shape[0] and col < C.shape[1]:
        tmp = 0.
        for k in range(A.shape[1]):
            tmp += A[row, k] * B[k, col]
        C[row, col] = tmp

- The host code creates and initiliazes the arrays `A` and `B`, then moves them to the device.
- It allocates space on the device for the result array. 
- Once the kernel has completed, the result array must be copied back to the host so that it can be displayed.

In [ ]:
blockdim = (16, 16)
griddim = (8, 8)

In [ ]:
tCudaMat1 = %timeit -o matrix_multiplication_cuda[griddim, blockdim](A, B, C)

In [ ]:
print("Speedup Regular: {}".format(tRegMat.best/tRegMat.best))
print("Speedup Numba:   {}".format(tRegMat.best/tNumMat.best))
print("Speedup CUDA 1:  {}".format(tRegMat.best/tCudaMat1.best))

#### Second GPU Kernel

There can be a faster way for the matrix multiplication with Cuda:

- Each thread block is responsible for computing a square sub-matrix of `C` and each thread for computing an element of the sub-matrix. 
- The sub-matrix is equal to the product of a square sub-matrix of `A` (`sA`) and a square sub-matrix of `B` (`sB`). 
- In order to fit into the device resources, the two input matrices are divided into as many square sub-matrices of dimension `TPB` as necessary, and the result computed as the sum of the products of these square sub-matrices.
- Each product is performed by first loading `sA` and `sB` from global memory to shared memory, with one thread loading each element of each sub-matrix. 
- Once `sA` and sB have been loaded, each thread accumulates the result into a register (tmp). Once all the products have been calculated, the results are written to the matrix `C` in global memory.
- The number of global memory accesses is reduced since `A` is now only read `B.shape[1] / TPB` times and `B` is read `A.shape[0] / TPB` times.



![matmult2](https://nyu-cds.github.io/python-numba/fig/05-matmulshared.png)
Image Source: nyu-cds.github.io

In [ ]:
from numba import cuda, float32

# Controls threads per block and shared memory usage.
# The computation will be done on blocks of TPBxTPB elements.
TPB = 16

@cuda.jit
def matrix_multiplication_cuda_fast(A, B, C):
    # Define an array in the shared memory
    # The size and type of the arrays must be known at compile time
    sA = cuda.shared.array(shape=(TPB, TPB), dtype=float32)
    sB = cuda.shared.array(shape=(TPB, TPB), dtype=float32)

    x, y = cuda.grid(2)

    tx = cuda.threadIdx.x
    ty = cuda.threadIdx.y
    bpg = cuda.gridDim.x    # blocks per grid

    if x >= C.shape[0] and y >= C.shape[1]:
        # Quit if (x, y) is outside of valid C boundary
        return

    # Each thread computes one element in the result matrix.
    # The dot product is chunked into dot products of TPB-long vectors.
    tmp = 0.
    for i in range(bpg):
        # Preload data into shared memory
        sA[tx, ty] = A[x, ty + i * TPB]
        sB[tx, ty] = B[tx + i * TPB, y]

        # Wait until all threads finish preloading
        cuda.syncthreads()

        # Computes partial product on the shared memory
        for j in range(TPB):
            tmp += sA[tx, j] * sB[j, ty]

        # Wait until all threads finish computing
        cuda.syncthreads()

    C[x, y] = tmp

In [ ]:
blockdim = (16, 16)
griddim = (8, 8)

In [ ]:
tCudaMat2 = %timeit -o matrix_multiplication_cuda_fast[griddim, blockdim](A, B, C)

In [ ]:
print("Speedup Regular: {}".format(tRegMat.best/tRegMat.best))
print("Speedup Numba:   {}".format(tRegMat.best/tNumMat.best))
print("Speedup CUDA 1:  {}".format(tRegMat.best/tCudaMat1.best))
print("Speedup CUDA 2:  {}".format(tRegMat.best/tCudaMat2.best))

### <font color="green">Example:</font> Mandelbrot Fractal 

In [ ]:
import matplotlib.pyplot as plt
from timeit import default_timer as timer

**Pure Python**

In [ ]:
# color function for point at (x, y)
def mandel(x, y, max_iters):
    c = complex(x, y)
    z = 0.0j
    for i in range(max_iters):
        z = z*z + c
        if z.real*z.real + z.imag*z.imag >= 4:
            return i
    return max_iters

In [ ]:
def create_fractal(xmin, xmax, ymin, ymax, image, iters):
    height, width = image.shape

    pixel_size_x = (xmax - xmin)/width
    pixel_size_y = (ymax - ymin)/height

    for x in range(width):
        real = xmin + x*pixel_size_x
        for y in range(height):
            imag = ymin + y*pixel_size_y
            color = mandel(real, imag, iters)
            image[y, x]  = color

In [ ]:
gimage = np.zeros((1024, 1536), dtype=np.uint8)
xmin, xmax, ymin, ymax = np.array([-2.0, 1.0, -1.0, 1.0]).astype('float32')
iters = 50

start = timer()
create_fractal(xmin, xmax, ymin, ymax, gimage, iters)
dt_py = timer() - start

print("Mandelbrot created on CPU in {} s".format(dt_py))
plt.imshow(gimage);

**Numba Code**

In [ ]:
mandel_numba = jit(nb.uint32(nb.float32, nb.float32, nb.uint32))(mandel)

In [ ]:
@jit
def create_fractal_numba(xmin, xmax, ymin, ymax, image, iters):
    height, width = image.shape

    pixel_size_x = (xmax - xmin)/width
    pixel_size_y = (ymax - ymin)/height

    for x in range(width):
        real = xmin + x*pixel_size_x
        for y in range(height):
            imag = ymin + y*pixel_size_y
            color = mandel_numba(real, imag, iters)
            image[y, x]  = color

In [ ]:
gimage = np.zeros((1024, 1536), dtype=np.uint8)
xmin, xmax, ymin, ymax = np.array([-2.0, 1.0, -1.0, 1.0]).astype('float32')
iters = 50

start = timer()
create_fractal_numba(xmin, xmax, ymin, ymax, gimage, iters)
dt_numba = timer() - start

print("Mandelbrot created on CPU in {} s".format(dt_numba))
plt.imshow(gimage);

**CUDA Code**

In [ ]:
mandel_gpu = cuda.jit(restype=nb.uint32, 
                      argtypes=[nb.float32, nb.float32, nb.uint32], 
                      device=True)(mandel)

In [ ]:
@cuda.jit(argtypes=[nb.float32, nb.float32, nb.float32, nb.float32, nb.uint8[:,:], nb.uint32])
def create_fractal_kernel(xmin, xmax, ymin, ymax, image, iters):
    height, width = image.shape

    pixel_size_x = (xmax - xmin)/width
    pixel_size_y = (ymax - ymin)/height

    startX, startY = cuda.grid(2)
    gridX = cuda.gridDim.x * cuda.blockDim.x # stride in x
    gridY = cuda.gridDim.y * cuda.blockDim.y # stride in y

    for x in range(startX, width, gridX):
        real = xmin + x*pixel_size_x
        for y in range(startY, height, gridY):
            imag = ymin + y*pixel_size_y
            color = mandel_gpu(real, imag, iters)
            image[y, x]  = color

In [ ]:
gimage = np.zeros((1024, 1536), dtype=np.uint8)
blockdim = (32, 8)
griddim = (32, 16)
xmin, xmax, ymin, ymax = np.array([-2.0, 1.0, -1.0, 1.0]).astype('float32')
iters = 50

start = timer()
d_image = cuda.to_device(gimage)
create_fractal_kernel[griddim, blockdim](xmin, xmax, ymin, ymax, d_image, iters)
d_image.to_host()
dt_cuda = timer() - start

print("Mandelbrot created on GPU in {} s".format(dt_cuda))
plt.imshow(gimage);

In [ ]:
print("Speedup CPU:    {}".format(dt_py/dt_py))
print("Speedup Numba:  {}".format(dt_py/dt_numba))
print("Speedup CUDA:   {}".format(dt_py/dt_cuda))

# [cuDF and Dask-cuDF](https://docs.rapids.ai/api/cudf/stable/10min.html)

#### Install RAPIDS

- Install most recent Miniconda release compatible with Google Colab's Python install (3.7.10)
- Removes incompatible files
- Install RAPIDS' current stable version of its libraries, including:
   - cuDF
   - cuML
   - cuGraph
   - cuSpatial
   - cuSignal
   - xgboost
- Set necessary environment variables
- Copy RAPIDS .so files into current working directory, a workaround for conda/colab interactions

In [ ]:
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!bash rapidsai-csp-utils/colab/rapids-colab.sh stable

import sys, os

dist_package_index = sys.path.index('/usr/local/lib/python3.7/dist-packages')
sys.path = sys.path[:dist_package_index] + ['/usr/local/lib/python3.7/site-packages'] + sys.path[dist_package_index:]
sys.path
exec(open('rapidsai-csp-utils/colab/update_modules.py').read(), globals())

In [ ]:
import cudf
import cupy as cp
import dask_cudf
import numpy as np

print('Dask cuDF Version:', dask_cudf.__version__)

Use `cudf` to create a dataframe and perform operations:

In [ ]:
num_rows = 5000000
df = cudf.DataFrame({'X':np.random.randint(1000, size=num_rows),
                     'Y':np.random.randint(1000, size=num_rows)})
df

In [ ]:
def add_squares(df):
    return df.X**2 + df.Y**2

In [ ]:
%%time

df['add_squares'] = add_squares(df)

In [ ]:
df

Do the same as above using `dask_cudf`:

In [ ]:
ddf = dask_cudf.from_cudf(df, npartitions=2)
ddf.head()

In [ ]:
%%time

ddf['z'] = add_squares(ddf).compute()

In [ ]:
ddf.head()

**Time Series Data**

`DataFrames` supports `datetime` typed columns, which allow users to interact with and filter data based on specific timestamps.

In [ ]:
import datetime as dt
import pandas as pd

date_df = cudf.DataFrame()
date_df['date'] = pd.date_range('01/05/1980', periods=15000, freq='D')
date_df['value'] = cp.random.sample(len(date_df))
date_df

In [ ]:
search_date1 = dt.datetime.strptime('2001-09-11', '%Y-%m-%d')
search_date2 = dt.datetime.strptime('2019-11-23', '%Y-%m-%d')

In [ ]:
%%time

date_df.query('date >= @search_date1 and date <= @search_date2')

In [ ]:
date_ddf = dask_cudf.from_cudf(date_df, npartitions=4)

In [ ]:
date_ddf.head()

In [ ]:
%%time

date_ddf.query('date >= @search_date1 and date <= @search_date2', 
               local_dict={'search_date1':search_date1, 
                           'search_date2':search_date2}).compute()

### <font color="green"> NYC Flights Dataset</font>

Data is specific to flights (in 1990's) out of the three airports in the New York City area.


Download the remote data:

In [ ]:
import urllib.request

print("\t Downloading NYC dataset...", end="\n", flush=True)

url = "https://storage.googleapis.com/dask-tutorial-data/nycflights.tar.gz"
filename, header = urllib.request.urlretrieve(url, "nycflights.tar.gz")

print("\t Done!", flush=True)

Extract the .csv files from the tar file:

In [ ]:
import tarfile

with tarfile.open(filename, mode="r:gz") as flights:
     flights.extractall("data/")

In [ ]:
!ls -lrt data/nycflights

Read all the files at once:

In [ ]:
import os

df = dask_cudf.read_csv(os.path.join("data", "nycflights", "*.csv")) 
df

- The representation of the dataframe object contains no data. 
- `cudf.read_csv` reads in the entire file before inferring datatypes.
- `dask_cudf.dataframe.read_csv` only reads in a sample from the beginning of the file (or first file). These inferred datatypes are then enforced when reading all partitions.

We can display the first few rows:

In [ ]:
df.head()

If we display the last few rows, we have a problem:

In [ ]:
df.tail()

- There is an issue with the data types of few columns.
- The datatypes inferred in the sample are incorrect.
- We can fix it by reading the files again and specify the appropriate data types.

In [ ]:
df.columns

In [ ]:
col_type = {
    'Year': int, 'Month': int, 'DayofMonth': int, 'DayOfWeek': int, 
    'DepTime': float, 'CRSDepTime': int, 'ArrTime': float , 
    'CRSArrTime': int, 'UniqueCarrier': str, 'FlightNum': int, 
    'TailNum': str, 'ActualElapsedTime': float, 
    'CRSElapsedTime': float, 'AirTime': float, 'ArrDelay': float,
    'DepDelay': float, 'Origin': str, 'Dest': str, 'Distance': int, 
    'TaxiIn': int, 'TaxiOut': int, 'Cancelled': bool, 'Diverted': bool
}

In [ ]:
df = dask_cudf.read_csv(os.path.join("data", "nycflights", "*.csv"), 
                        dtype=col_type)

In [ ]:
df.head()

### <font color="blue">Perform Operations as with `CuDF DataFrames`</font>

**Maximum value of a column**:

- We now want to compute the maximum of the `DepDelay` column.
- With `CuDF`, we would loop over each file to find the individual maximums, then find the final maximum over all the individual maximums.
- `dask_cudf.dataframe` allows us to write pandas-like code that operates on large than memory datasets in parallel.

In [ ]:
df.DepDelay.max().visualize()

In [ ]:
%time df.DepDelay.max().compute()

If we do the same thing in `CuDF`, we will have:

In [ ]:
%%time

import glob

list_files = glob.glob("data/nycflights/*csv")
   
maxes = list()
for file_name in list_files:
    pddf = cudf.read_csv(file_name)
    maxes.append(pddf.DepDelay.max())

final_max = max(maxes)

print("Final Maximum: ", max(maxes))

#### Other Operations

Number of departures from each airport:

In [ ]:
df.groupby("Origin").Origin.count().compute()

Number of non-cancelled flights:

In [ ]:
len(df[~df.Cancelled])

Number of non-cancelled flights were taken from each airport:

In [ ]:
df[~df.Cancelled].groupby('Origin').Origin.count().compute()

Group by destinations and count:

In [ ]:
df.groupby("Dest").count().compute()

In [ ]:
df.groupby("Dest")["ArrDelay"].mean().compute()

In [ ]:
df[df.ArrDelay+df.DepDelay>30.0].groupby("Dest").Dest.count().compute()

### Exercise 

- Consider the CuDF code below that computes the mean departure delay per airport. 
- Parallelize the code using Dask-CuDF.

In [ ]:
%%time 

sum_delays = list()
count_delays = list()

for file_name in list_files:
    pddf = cudf.read_csv(file_name)
    by_origin = pddf.groupby('Origin')
    loc_total = by_origin.DepDelay.sum()
    loc_count = by_origin.DepDelay.count()
    sum_delays.append(loc_total)
    count_delays.append(loc_count)

total_delays = sum(sum_delays)
n_flights = sum(count_delays)
mean_delays = total_delays / n_flights
print("Mean delays: {}".format(mean_delays))

<details><summary><b><font color="violet" size=4>Click here to access the solution</font> </b></summary>
<p>

```python
%%time 
    
df.groupby("Origin").DepDelay.mean().compute()   
```
    
</p>
</details>